In [199]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
from rake_nltk import Rake
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import streamlit as st

In [201]:
df_anime = pd.read_csv("../data/df_anime_export.csv")

## View Resetter

In [152]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [189]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

## Cleaning anime_df a bit more 

In [202]:
df_anime.head(5)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ..."
1,1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,"music accompanies path human metronome , prodi..."
2,2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,"abyss—a gaping chasm stretching depth earth , ..."
3,3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,"`` order something obtained , something equal ..."
4,4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,helping revive legendary vampire kiss-shot ace...


In [203]:
for idx, genre in enumerate(df_anime['genre']):
    df_anime.at[idx, 'genre'] = genre.replace('Martial Arts', 'Martial_Arts')
    df_anime.at[idx, 'genre'] = genre.replace('Super Power', 'Super_Power')
    df_anime.at[idx, 'genre'] = genre.replace('Slice of Life', 'Slice_of_Life')

In [204]:
#clean genre so it can be used in the rec system
for idx, row_genre in enumerate(df_anime['genre']):
    row_genre = row_genre.replace("'",'')
    row_genre = row_genre.replace(",",'')
    row_genre = row_genre.replace("]","")
    row_genre = row_genre.replace("[","")
    df_anime['genre'].iloc[idx] = row_genre
df_anime['genre']

C:\Users\samue\AppData\Local\Temp\ipykernel_22640\1510632593.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anime['genre'].iloc[idx] = row_genre


0                       Comedy Sports Drama School Shounen
1                       Drama Music Romance School Shounen
2                   Sci-Fi Adventure Mystery Drama Fantasy
3        Action Military Adventure Comedy Drama Magic F...
4                      Action Mystery Supernatural Vampire
                               ...                        
16209       Action Comedy Super Power Martial Arts Shounen
16210                    Slice_of_Life Comedy Supernatural
16211                         Slice_of_Life Comedy Shounen
16212                                               Action
16213            Comedy Drama Romance School Slice_of_Life
Name: genre, Length: 16214, dtype: object

## Begin recc model with preprocessed synopsis

In [205]:
synopsis = df_anime['synopsis_processed']
genre = df_anime['genre']
title = df_anime['title']

In [206]:
df_touse = df_anime[["synopsis_processed","genre","title","popularity"]]

In [207]:
# Calculate the minimum and maximum values of the 'popularity' column
min_popularity = df_touse['popularity'].min()
max_popularity = df_touse['popularity'].max()

# Perform min-max scaling on the 'popularity' column
df_touse['popularity_normalized'] = (df_touse['popularity'] - min_popularity) / (max_popularity - min_popularity)

C:\Users\samue\AppData\Local\Temp\ipykernel_22640\3110732407.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_touse['popularity_normalized'] = (df_touse['popularity'] - min_popularity) / (max_popularity - min_popularity)


In [208]:
df_touse.head(1)

,synopsis_processed,genre,title,popularity,popularity_normalized
0,"following participation inter-high , karasuno ...",Comedy Sports Drama School Shounen,Haikyuu!! Second Season,141,0.00857


In [248]:
#extracting keywords for recommender
rake = Rake()
words = []
for plot in synopsis:
    rake.extract_keywords_from_text(str(plot))
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    words.append(keywords_i_string)
df_touse['words'] = words

#combining to use in vectorizer
df_touse['all_words'] = df_touse['genre'] + df_touse['words']
vectorizer = CountVectorizer()
vectorized_bag_of_words = vectorizer.fit_transform(df_touse['all_words'])
vectorized_bag_of_words = vectorized_bag_of_words.toarray()

C:\Users\samue\AppData\Local\Temp\ipykernel_22640\1931016245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_touse['all_words'] = df_touse['genre'] + df_touse['words']


In [233]:
from scipy.sparse import csr_matrix, hstack

In [244]:
def vectorize_genre_and_words(genre_column, synopsis, genre_weight=2.0, words_weight=1.0):
    #extracting keywords for recommender
    rake = Rake()
    words = []
    for plot in synopsis:
        rake.extract_keywords_from_text(str(plot))
        keywords_i = rake.get_ranked_phrases()
        keywords_i_string = ""
        for keyword in keywords_i:
            keywords_i_string = keywords_i_string + " " + keyword
        words.append(keywords_i_string)
    #temp_df['words'] = words
    # Ended up not needing above
    
    #Adjust weight as needed
    # Combine genre and words into a single column
    #combined_column = genre_column + words
    # This ^ ultimately ended up not being used, will leave in for legacy

    # Create a TF-IDF vectorizer
    vectorizer_words = TfidfVectorizer()
    vectorized_words = vectorizer_words.fit_transform(words)
                                                    #try temp_df['words'] if words dont work

    # Create a TF-IDF vectorizer
    vectorizer_genre = TfidfVectorizer()
    vectorized_genre = vectorizer_genre.fit_transform(genre_column)

    # Get the number of genres
    num_genre_features = vectorized_genre.shape[1]

    # Apply the weights to the genre and words vectors
    weighted_vectorized_genre = vectorized_genre.multiply(genre_weight)
    weighted_vectorized_words = vectorized_words.multiply(words_weight)

    # Combine the genre and words vectors
    vectorized_combined = hstack([weighted_vectorized_genre, weighted_vectorized_words])

    # Convert to csr_matrix
    vectorized_combined = csr_matrix(vectorized_combined)

    # Convert to array
    vectorized_combined = vectorized_combined.toarray()

    return vectorized_combined

In [212]:
#trying to not include certain genres
def recommend(show_title, n_recom, vectorized_bag_of_words, title, not_genres=None):
    # Find the indices of the show title in the DataFrame
    show_indices = np.where(df_touse["title"] == show_title)[0]
    if len(show_indices) == 0:
        return []  # Return an empty list if the show title is not found

    # Calculate the similarity matrix
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[show_indices, :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title
    similarity_dataframe = similarity_dataframe.iloc[:, 0]

    # Filter out shows with specified genres
    if not_genres:
        similarity_dataframe = similarity_dataframe[~genre.isin(not_genres)]

    # Sort the similarity scores in descending order
    similarity_dataframe = similarity_dataframe.sort_values(ascending=False)

    # Drop duplicates from the similarity scores
    similarity_dataframe = similarity_dataframe.drop_duplicates()

    # Return the recommended show titles
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [213]:
def recommend(show_title, n_recom, vectorized_bag_of_words, title, not_genres=None):
    # Find the indices of the show title in the DataFrame
    show_indices = np.where(title == show_title)[0]
    if len(show_indices) == 0:
        return []  # Return an empty list if the show title is not found

    # Calculate the similarity matrix
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[show_indices, :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    
    # Filter out shows with specified genres
    genre = df_anime['genre']  # Assuming 'df_anime' is the correct DataFrame
    if not_genres:
        similarity_dataframe = similarity_dataframe[~genre.isin(not_genres)]

    # Sort the similarity scores in descending order
    similarity_dataframe = similarity_dataframe.iloc[:, 0].sort_values(ascending=False)

    # Drop duplicates from the similarity scores
    similarity_dataframe = similarity_dataframe.drop_duplicates()

    # Get the recommended show indices
    recommended_indices = list(similarity_dataframe.index)[1:n_recom + 1]
    
    # Get the corresponding show titles
    recommended_titles = title[recommended_indices].tolist()

    # Return a list of tuples with show titles and their corresponding unique ids
    recommended_shows = list(zip(recommended_titles, recommended_indices))
    return recommended_shows

In [214]:
#just removes similar titles
def recommend_2(show_title, n_recom, vectorized_bag_of_words, title):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(title == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title 
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    
    recommended_titles = list(similarity_dataframe.index)
    recommended_titles = [title for title in recommended_titles if show_title not in title]  # Remove titles containing the show_title input
    
    return recommended_titles[:n_recom]

In [215]:
# Regular recommender, returns similar titles
def recommend_3(show_title, n_recom, vectorized_bag_of_words, title):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(title == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title 
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [246]:
#consider popularity too
# Considers ALL titles similar to input + popularity
def recommend_4(show_title, n_recom, vectorized_bag_of_words, title, df):
    show_indices = np.where(title == show_title)[0]
    if len(show_indices) == 0:
        print(f"Show title '{show_title}' not found in the DataFrame.")
        return []  # Return an empty list if the show title is not found

    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(title == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title
    similarity_dataframe = similarity_dataframe.iloc[:, [0]]

    # Calculate the popularity weights
    max_popularity = df['popularity_normalized'].max()
    
    popularity_weights = (1 - df['popularity_normalized']) / max_popularity

    #print(popularity_weights)

    # Multiply the similarity scores by popularity weights
    similarity_dataframe *= popularity_weights

    similarity_dataframe = similarity_dataframe.sort_values(by = 0, ascending=False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()

    # Find similar titles using fuzzy matching
    similar_titles = []
    for title in similarity_dataframe.index:
        if fuzz.partial_ratio(show_title, title) >= 30:  # Set a threshold for similarity
            if show_title not in title:
                similar_titles.append(title)

    return similar_titles[:n_recom]

In [196]:
#attempt to weigh synopsis
# this is most likely dead end
def recommend_5(show_title, n_recom, vectorized_bag_of_words, title, df, synopsis_weight=0.5, genre_weight=0.5):
    show_indices = np.where(title == show_title)[0]
    if len(show_indices) == 0:
        print(f"Show title '{show_title}' not found in the DataFrame.")
        return []  # Return an empty list if the show title is not found

    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(title == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title
    similarity_dataframe = similarity_dataframe.iloc[:, [0]]

    # Calculate the popularity weights
    max_popularity = df['popularity_normalized'].max()
    popularity_weights = (max_popularity - df['popularity_normalized']) / max_popularity

    # Multiply the similarity scores by popularity weights
    similarity_dataframe *= popularity_weights

    # Calculate the weights for synopsis and genre components
    total_weight = synopsis_weight + genre_weight
    synopsis_weight /= total_weight
    genre_weight /= total_weight

    # Apply the weights to the similarity scores
    similarity_dataframe *= synopsis_weight
    similarity_dataframe += genre_weight * df['genre_weight_column']  # Replace 'genre_weight_column' with the actual column name containing genre weights

    similarity_dataframe = similarity_dataframe.sort_values(by=0, ascending=False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()

    # Find similar titles using fuzzy matching
    similar_titles = []
    for title in similarity_dataframe.index:
        if fuzz.partial_ratio(show_title, title) >= 30:  # Set a threshold for similarity
            if show_title not in title:
                similar_titles.append(title)

    return similar_titles[:n_recom]

In [249]:
title = df_touse['title']

vectorized_combined = vectorize_genre_and_words(df_touse['genre'], df_touse['synopsis_processed'], genre_weight=2.0, words_weight=1.0)


In [250]:
#broken genre filter
dont_include = ["Comedy"]
recommend("Fullmetal Alchemist",10, vectorized_bag_of_words, title, dont_include)

MemoryError: Unable to allocate 4.93 GiB for an array with shape (16214, 40848) and data type float64

In [251]:
#removes similar titles
recommend_2("Fullmetal Alchemist",10, vectorized_combined, title)


MemoryError: Unable to allocate 4.94 GiB for an array with shape (16214, 40889) and data type float64

In [235]:
#default just give back all
recommend_3("Fullmetal Alchemist",10, vectorized_combined, title)

['Fullmetal Alchemist: Brotherhood',
 'Fullmetal Alchemist: The Sacred Star of Milos',
 'Fullmetal Alchemist: Brotherhood Specials',
 'Fullmetal Alchemist: Reflections',
 'Tales of Vesperia: The First Strike',
 'Tide-Line Blue',
 'Fairy Tail Movie 1: Houou no Miko',
 'Katsute Kami Datta Kemono-tachi e',
 'Fairy Tail',
 'Fairy Tail x Rave']

In [247]:
#also considering popularity
recommend_4("Fullmetal Alchemist",10, vectorized_combined, title, df_touse)

['Tales of Vesperia: The First Strike',
 'Katsute Kami Datta Kemono-tachi e',
 'Fairy Tail',
 'Fairy Tail: Final Series',
 'Densetsu no Yuusha no Densetsu',
 'Fire Emblem',
 'Dragon Quest: Dai no Daibouken Buchiyabure!! Shinsei 6 Daishougun',
 'Dragon Quest: Dai no Daibouken Tachiagare!! Aban no Shito',
 'Chain Chronicle: Haecceitas no Hikari Part 2',
 'Slayers Great']

In [197]:
recommend_5("Fullmetal Alchemist",10, vectorized_bag_of_words, title, df_touse, synopsis_weight=0.5, genre_weight=0.5)

KeyError: 'genre_weight_column'

# Feature Engineered Recommendations

In [69]:
# Calculate the minimum and maximum values of the 'popularity' column
min_popularity = df_anime['popularity'].min()
max_popularity = df_anime['popularity'].max()

# Perform min-max scaling on the 'popularity' column
df_anime['popularity_normalized'] = (df_anime['popularity'] - min_popularity) / (max_popularity - min_popularity)

In [70]:
df_anime_sorted = df_anime.sort_values('popularity', ascending = True)

df_anime_sorted.head(5)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
740,740,1535,Death Note,"A shinigami, as a god of death, can kill any p...",Mystery Police Psychological Supernatural Thri...,"Oct 4, 2006 to Jun 27, 2007",37.0,1871043,1,52.0,8.65,"shinigami , god death , kill person—provided s...",0.000000
681,681,16498,Shingeki no Kyojin,"Centuries ago, mankind was slaughtered to near...",Action Military Mystery Super Power Drama Fant...,"Apr 7, 2013 to Sep 29, 2013",25.0,1754979,2,111.0,8.47,"century ago , mankind slaughtered near extinct...",0.000061
7881,8409,11757,Sword Art Online,"In the year 2022, virtual reality has progress...",Action Adventure Fantasy Game Romance,"Jul 8, 2012 to Dec 23, 2012",25.0,1657823,3,1771.0,7.49,"year 2022 , virtual reality progressed leap bo...",0.000122
3,3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",Action Military Adventure Comedy Drama Magic F...,"Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,"`` order something obtained , something equal ...",0.000184
16,16,30276,One Punch Man,The seemingly ordinary and unimpressive Saitam...,Action Sci-Fi Comedy Parody Super Power Supern...,"Oct 5, 2015 to Dec 21, 2015",12.0,1475219,5,50.0,8.68,seemingly ordinary unimpressive saitama rather...,0.000245


In [71]:

df_top_1000 = df_anime_sorted.head(1000)
df_top_1000


,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
740,740,1535,Death Note,"A shinigami, as a god of death, can kill any p...",Mystery Police Psychological Supernatural Thri...,"Oct 4, 2006 to Jun 27, 2007",37.0,1871043,1,52.0,8.65,"shinigami , god death , kill person—provided s...",0.000000
681,681,16498,Shingeki no Kyojin,"Centuries ago, mankind was slaughtered to near...",Action Military Mystery Super Power Drama Fant...,"Apr 7, 2013 to Sep 29, 2013",25.0,1754979,2,111.0,8.47,"century ago , mankind slaughtered near extinct...",0.000061
7881,8409,11757,Sword Art Online,"In the year 2022, virtual reality has progress...",Action Adventure Fantasy Game Romance,"Jul 8, 2012 to Dec 23, 2012",25.0,1657823,3,1771.0,7.49,"year 2022 , virtual reality progressed leap bo...",0.000122
3,3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",Action Military Adventure Comedy Drama Magic F...,"Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,"`` order something obtained , something equal ...",0.000184
16,16,30276,One Punch Man,The seemingly ordinary and unimpressive Saitam...,Action Sci-Fi Comedy Parody Super Power Supern...,"Oct 5, 2015 to Dec 21, 2015",12.0,1475219,5,50.0,8.68,seemingly ordinary unimpressive saitama rather...,0.000245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861,6234,36286,Re:Zero kara Hajimeru Isekai Seikatsu - Memory...,Subaru and friends finally get a moment of pea...,Psychological Drama Thriller Fantasy,"Oct 6, 2018",1.0,114324,998,1394.0,7.61,"subaru friend finally get moment peace , subar...",0.061027
7623,8151,32526,Love Live! Sunshine!!,"Chika Takami, a self-proclaimed normal girl, h...",Music Slice_of_Life School,"Jul 2, 2016 to Sep 24, 2016",13.0,113469,999,1799.0,7.48,"chika takami , self-proclaimed normal girl , n...",0.061088
6263,6719,4182,Darker than Black: Kuro no Keiyakusha - Sakura...,"A side-story of Darker than Black , also call...",Sci-Fi Comedy Parody,"Mar 26, 2008",1.0,112545,1000,1404.0,7.60,"side-story darker black , also called episode ...",0.061150
6996,7500,28391,Ao no Kanata no Four Rhythm,With the invention of anti-gravitational shoes...,Sci-Fi Sports Drama School,"Jan 12, 2016 to Mar 29, 2016",12.0,112517,1002,4532.0,6.81,invention anti-gravitational shoe known grav-s...,0.061272


In [110]:
df_touse_t1000 = df_top_1000[["synopsis_processed","genre","title","popularity"]]
df_touse_t1000

,synopsis_processed,genre,title,popularity
740,"shinigami , god death , kill person—provided s...",Mystery Police Psychological Supernatural Thri...,Death Note,1
681,"century ago , mankind slaughtered near extinct...",Action Military Mystery Super Power Drama Fant...,Shingeki no Kyojin,2
7881,"year 2022 , virtual reality progressed leap bo...",Action Adventure Fantasy Game Romance,Sword Art Online,3
3,"`` order something obtained , something equal ...",Action Military Adventure Comedy Drama Magic F...,Fullmetal Alchemist: Brotherhood,4
16,seemingly ordinary unimpressive saitama rather...,Action Sci-Fi Comedy Parody Super Power Supern...,One Punch Man,5
...,...,...,...,...
5861,"subaru friend finally get moment peace , subar...",Psychological Drama Thriller Fantasy,Re:Zero kara Hajimeru Isekai Seikatsu - Memory...,998
7623,"chika takami , self-proclaimed normal girl , n...",Music Slice_of_Life School,Love Live! Sunshine!!,999
6263,"side-story darker black , also called episode ...",Sci-Fi Comedy Parody,Darker than Black: Kuro no Keiyakusha - Sakura...,1000
6996,invention anti-gravitational shoe known grav-s...,Sci-Fi Sports Drama School,Ao no Kanata no Four Rhythm,1002


In [112]:
df_touse_t1000 = df_top_1000[["synopsis_processed","genre","title","popularity"]]

synopsis_t1000 = df_touse_t1000['synopsis_processed']
genre_t1000 = df_touse_t1000['genre']
title_t1000 = df_touse_t1000['title']

rake = Rake()
words = []
for plot in synopsis_t1000:
    rake.extract_keywords_from_text(str(plot))
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    words.append(keywords_i_string)
df_touse_t1000['words'] = words

df_touse_t1000['all_words'] = df_touse_t1000['genre'] + df_touse_t1000['words']

vectorizer = CountVectorizer()
vectorized_t1000_words = vectorizer.fit_transform(df_touse_t1000['all_words'])
vectorized_t1000_words = vectorized_t1000_words.toarray()


C:\Users\samue\AppData\Local\Temp\ipykernel_22640\3820357729.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_touse_t1000['words'] = words
C:\Users\samue\AppData\Local\Temp\ipykernel_22640\3820357729.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_touse_t1000['all_words'] = df_touse_t1000['genre'] + df_touse_t1000['words']


In [113]:

recommend("Fullmetal Alchemist",10, vectorized_t1000_words, title_t1000)

['Fullmetal Alchemist',
 'Fullmetal Alchemist: The Sacred Star of Milos',
 'Fullmetal Alchemist: The Conqueror of Shamballa',
 'Oniichan dakedo Ai sae Areba Kankeinai yo ne!',
 'No Game No Life: Zero',
 'Joker Game',
 'Fairy Tail Movie 1: Houou no Miko',
 'Casshern Sins',
 'Junjou Romantica',
 'InuYasha: Kanketsu-hen']

In [114]:

recommend_2("Fullmetal Alchemist: Brotherhood",10, vectorized_t1000_words, title_t1000)

['Fullmetal Alchemist',
 'Fullmetal Alchemist: The Sacred Star of Milos',
 'Fullmetal Alchemist: The Conqueror of Shamballa',
 'Oniichan dakedo Ai sae Areba Kankeinai yo ne!',
 'No Game No Life: Zero',
 'Joker Game',
 'Fairy Tail Movie 1: Houou no Miko',
 'Casshern Sins',
 'Junjou Romantica',
 'InuYasha: Kanketsu-hen']

In [129]:

recommend_3("Death Note",10, vectorized_t1000_words, title_t1000)

['Dragon Ball',
 'Dragon Ball GT',
 'Hagure Yuusha no Aesthetica',
 'Yu☆Gi☆Oh! Duel Monsters',
 'C: The Money of Soul and Possibility Control',
 'Naruto: Shippuuden',
 'Boku no Hero Academia 2nd Season',
 'Dragon Ball Super',
 'D.Gray-man Hallow',
 'Absolute Duo']

## Conclusion:
- At bare minimum, the first model will be the one used. Reduce dimensionality as much as possible
- Try doing PCA too later

In [21]:
show_list = df_anime['title'].to_list()

In [22]:
import re
from collections import defaultdict

# Assuming you have a list called "show_list" containing the show titles

# Create a defaultdict with a list as the default value
show_dict = defaultdict(list)

# Group the show titles by their first character (English alphabet or number)
for show_title in show_list:
    match = re.match(r'^[A-Za-z0-9]', show_title)  # Match English alphabet character or number at the beginning
    if match:
        starting_character = match.group()
        if starting_character.isalpha():
            starting_letter = starting_character.upper()
            show_dict[starting_letter].append(show_title)
        else:
            show_dict['Numbers'].append(show_title)

# Sort the dictionary by keys (starting letters or 'Numbers') in alphabetical order
sorted_show_dict = dict(sorted(show_dict.items()))

# Print the shows for each starting letter or 'Numbers'
for starting_letter, shows in sorted_show_dict.items():
    grouped_shows = ', '.join(shows)  # Join the show titles with a comma
    if starting_letter == 'Numbers':
        print(f"Shows starting with Numbers: {grouped_shows}")
    else:
        print(f"Shows starting with {starting_letter}: {grouped_shows}")


Shows starting with A: Asagao to Kase-san., Ansatsu Kyoushitsu 2nd Season: Kagaijugyou-hen, Akagami no Shirayuki-hime, Aggressive Retsuko (ONA) 2nd Season, Akachan to Boku, Aoki Hagane no Arpeggio: Ars Nova Cadenza, Aria The Avvenire, Azumanga Daioh, Aria The OVA: Arietta, Aria The Origination: Sono Choppiri Himitsu no Basho ni..., Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. Movie, Akagami no Shirayuki-hime 2nd Season, Akatsuki no Yona, Akatsuki no Yona OVA, Ashita no Joe, Akira, Aria The Natural, Ansatsu Kyoushitsu, Asobi Asobase, Angel Beats!, Ashita no Joe 2, Aria The Origination, Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai., Ansatsu Kyoushitsu 2nd Season, Araburu Kisetsu no Otome-domo yo., Arslan Senki (TV), Ao Haru Ride, Akagami no Shirayuki-hime: Nandemonai Takaramono, Kono Page, Arashi no Yoru ni, Arakawa Under the Bridge x Bridge, Aggressive Retsuko (ONA), Aoi Bungaku Series, Ajin 2nd Season, Ani ni Tsukeru Kusuri wa Nai! 2, Aikatsu Stars!, Aria The Ani

In [25]:
popular_list = df_anime_sorted['title'].to_list()
popular_list

['Death Note',
 'Shingeki no Kyojin',
 'Sword Art Online',
 'Fullmetal Alchemist: Brotherhood',
 'One Punch Man',
 'Tokyo Ghoul',
 'Steins;Gate',
 'No Game No Life',
 'Naruto',
 'Angel Beats!',
 'Code Geass: Hangyaku no Lelouch',
 'Boku no Hero Academia',
 'Mirai Nikki',
 'Toradora!',
 'Kimi no Na wa.',
 'Noragami',
 'Ao no Exorcist',
 'Sword Art Online II',
 'Naruto: Shippuuden',
 'Hunter x Hunter (2011)',
 'Fairy Tail',
 'Boku no Hero Academia 2nd Season',
 'Re:Zero kara Hajimeru Isekai Seikatsu',
 'Akame ga Kill!',
 'Bleach',
 'Elfen Lied',
 'Code Geass: Hangyaku no Lelouch R2',
 'Shigatsu wa Kimi no Uso',
 'Tengen Toppa Gurren Lagann',
 'Kill la Kill',
 'Shingeki no Kyojin Season 2',
 'Soul Eater',
 'Psycho-Pass',
 'Another',
 'One Piece',
 'Boku dake ga Inai Machi',
 'Fullmetal Alchemist',
 'Nanatsu no Taizai',
 'Cowboy Bebop',
 'Sen to Chihiro no Kamikakushi',
 'Durarara!!',
 'Highschool of the Dead',
 'Tokyo Ghoul √A',
 'Kiseijuu: Sei no Kakuritsu',
 'Clannad',
 'Kono Subarashii

In [144]:
string_to_check = "Love"
filtered_df = df_anime[df_anime['genre'].str.contains(string_to_check, case=False)]
filtered_df

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized


In [153]:
# Concatenate all genres into a single string
all_genres = ' '.join(df_anime['genre'])

# Split the string to get individual genres
individual_genres = all_genres.split()

# Count the occurrence of each genre
genre_counts = {}
for genre in individual_genres:
    genre_counts[genre] = genre_counts.get(genre, 0) + 1

# Sort the genres by count in descending order
sorted_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)

# Print unique genres and their counts in order
for genre, count in sorted_genres:
    print(f"{genre}: {count}")

Comedy: 5640
Action: 3572
Fantasy: 2971
Adventure: 2753
Sci-Fi: 2462
Drama: 2457
Kids: 2456
Shounen: 2006
Music: 1863
Romance: 1762
Slice_of_Life: 1744
School: 1567
Supernatural: 1358
Hentai: 1295
Mecha: 1060
Historical: 1050
Magic: 1013
Seinen: 805
Shoujo: 741
Ecchi: 740
Sports: 679
Mystery: 674
Parody: 619
Super: 600
Power: 600
Military: 541
Space: 475
Demons: 453
Horror: 434
Dementia: 402
Harem: 374
Martial: 367
Arts: 367
Game: 348
Psychological: 322
Police: 239
Samurai: 199
Ai: 164
Vampire: 135
Thriller: 120
Cars: 117
Josei: 93
Yaoi: 40
Yuri: 40
